**Getting Reddit Posts on Euphoria & Rue**
08/08/2022

*Season 1: June 16 - Aug 4, 2019*

*reason*: replicate upitt qualitative paper - same sampling criteria - more recent season.  
hope to have more relvant topics but similar topics they determined thematically  
sampling criteria:  
- overall r/television subr
- time frame  
- comments that mention 'rue' - as narrator and main character  

*analysis*: 
- topic modeling season 1 v 2
  - LDA
    - so far: getting comments for three recent top posts - 4 topics with 100 passes, no clear topics
    - hyperparameter tuning
      - coherence score/perplexity ...meh
  - transformer-based
    - `BERTopic`
      - embeddings: 
        - all-mpnet-base-v2
        - all-distilroberta-v1
        - all-MiniLM-L6-v2 - *default*
- compute valence scores for each comment and group by topic
  - read into mood shift of main character from season 1 to 2
- compare S1 to S2

In [10]:
# source api credentials from file
file = 'api-creds.py'
exec(open(file).read())

In [2]:
from psaw import PushshiftAPI
import pandas as pd
from datetime import datetime

# inclusive, make this the first day to collect yyyy,m,d
after = int(datetime(2019, 6, 16).timestamp()) -1

# exclusive, make this the day after the last day to collect
before = int(datetime(2019, 8, 5).timestamp())

api=PushshiftAPI()

# change to search_comments to collect comments instead of submissions
subs = api.search_submissions(after=after, before=before, q="rue", limit=None)

# process however you want - this loads the posts into a dataframe
df = pd.DataFrame([sub.d_ for sub in subs])

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [3]:
df.to_pickle('../dat/rue_s1_submissions.pkl')

In [4]:
s1_pids_df = df[['id', 'title', 'selftext', 'url', 'author', 'score', 'created_utc', 'num_comments']]
# convert created_utc to datetime
s1_pids_df['created_utc_real'] = pd.to_datetime(s1_pids_df['created_utc'], unit='s')

/var/folders/rd/p010xzd55nl_33_67s8769q00000gq/T/ipykernel_32176/3138543611.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1_pids_df['created_utc_real'] = pd.to_datetime(s1_pids_df['created_utc'], unit='s')


In [5]:
# filter for domain in self.euphoria
s1_pids_df = s1_pids_df[s1_pids_df['url'].str.contains('https://www.reddit.com/r/euphoria/')]

In [6]:
# concat comments and posts that are not None 
posts = s1_pids_df['selftext']
# filter out the None values
posts2 = pd.DataFrame([post for post in posts if post not in [None, '', 'None', '[deleted]', '[removed]']])

In [7]:
# using the id of a post, get the comments and store in a dataframe

def get_comments(submission_id):
    """
    Get comments from a submission
    """
    comments = []
    submission = reddit.submission(id = submission_id)
    submission.comments.replace_more(limit = None)
    for comment in submission.comments.list():
        comments.append({'body': comment.body, 'author': comment.author, 'score': comment.score, 'created_utc': comment.created_utc, 'id': comment.id})
    return comments

In [11]:
s1_comments = []
for post_id in s1_pids_df.id:
    s1_comments.append(get_comments(post_id))

In [13]:
# get the body of each comment
import math
def get_body(df, percent = 1):
    body_ls = []
    for i in range(len(df)):
        for j in range(len(df.iloc[i])):
            if type(df.iloc[i][j]) != type(None):
               body_ls.append(df.iloc[i][j]['body'])
    # adjust for percentage
    body_ls = body_ls[:math.floor(len(body_ls) * percent)]
    return body_ls 

In [14]:
# apply func to all the posts
s1_comments_df = pd.DataFrame(s1_comments)
s1_comment_body_df = pd.DataFrame(get_body(s1_comments_df, percent = 1))

In [15]:
s1_comments_df = pd.concat([s1_comment_body_df, posts2], ignore_index=True)

In [16]:
# save
s1_comments_df.to_pickle('../dat/s1_rue_comments.pkl')